In [ ]:
#r "nuget: Thinksharp.TimeFlow"
#r "nuget: Thinksharp.TimeFlow.Interactive"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

using Thinksharp.TimeFlow;

Installed Packages Thinksharp.TimeFlow, 0.21.0 Thinksharp.TimeFlow.Interactive, 0.21.0 XPlot.Plotly, 4.0.3 XPlot.Plotly.Interactive, 4.0.3

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `ThinkSharp.TimeFlow.Interactive.dll`

TimeFlow formatter registered.

---

## Time Series Creation

Time series can be created using the available methods of the static **TimeSeries.Factory** property.
The factory is of type **ITimeSeriesFactory**, which can be extended via extension methods. Therefore third party libraries may provide advanced factory methods or user can write it's own use-case specific ones.

### TimeSeries.Factory.FromValue

In [ ]:
// FromValue creates a time series with a defined value for each time point:
var ts = TimeSeries.Factory.FromValue(10,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day);

ts

2020-12-31 23:00:00Z,10
2021-01-01 23:00:00Z,10
2021-01-02 23:00:00Z,10
2021-01-03 23:00:00Z,10
2021-01-04 23:00:00Z,10


### TimeSeries.Factory.FromValues

In [ ]:
// FromValues creates a time series from an enumerable of values:
var values = new decimal?[] {1, 2, 3, 4, 5, 6, 7, 8, 9};
var ts = TimeSeries.Factory.FromValues(values,
    new DateTime(2021, 01, 01), // start
    Period.Day);

ts

2020-12-31 23:00:00Z,1
2021-01-01 23:00:00Z,2
2021-01-02 23:00:00Z,3
2021-01-03 23:00:00Z,4
2021-01-04 23:00:00Z,5
2021-01-05 23:00:00Z,6
2021-01-06 23:00:00Z,7
2021-01-07 23:00:00Z,8
2021-01-08 23:00:00Z,9


### TimeSeries.Factory.FromGenerator

In [ ]:
// FromGenerator creates a time series with a value created as function of the time point:
var ts = TimeSeries.Factory.FromGenerator(
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day,
    tp => tp.Day); // tp: time point as DateTimeOffset

ts

2020-12-31 23:00:00Z,1
2021-01-01 23:00:00Z,2
2021-01-02 23:00:00Z,3
2021-01-03 23:00:00Z,4
2021-01-04 23:00:00Z,5


---
## Time Series Transformation

### Combining Time Series via Operators

In [ ]:
// time series can be combined using default operators:
var a = TimeSeries.Factory.FromValue(10,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day);

var b = TimeSeries.Factory.FromGenerator(
    new DateTime(2021, 01, 01), // start   
    new DateTime(2021, 01, 05), // end 
    Period.Day,
    tp => tp.Day);

var tf = new TimeFrame();
tf["a"] = a;
tf["b"] = b;
tf["a + b"] = a + b;
tf["a - b"] = a - b;
tf["a * b"] = a * b;
tf["a / b"] = a / b;
tf["a * 12"] = a * 12;
tf


2020-12-31 23:00:00Z,10,1,11,9,10,10,120
2021-01-01 23:00:00Z,10,2,12,8,20,5,120
2021-01-02 23:00:00Z,10,3,13,7,30,3.3333333333333333333333333333,120
2021-01-03 23:00:00Z,10,4,14,6,40,2.5,120


### Combining Time Series via Methods

In [ ]:
var a = TimeSeries.Factory.FromValue(1,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day);

var b = TimeSeries.Factory.FromValue(2,
  new DateTime(2021, 01, 03), // start
  new DateTime(2021, 01, 07), // end
  Period.Day);

var tf = new TimeFrame();
tf["a"] = a;
tf["b"] = b;
tf["apply * 2"] = a.Apply(value => value * 2);

// join left produces a time series with the same time points as the left time series.
// note that nulls will be evaluated to null
tf["JoinLeft r + l"] = a.JoinLeft(b, (l, r) => l + r);
// Use pre defined JoinOperation to ignore nulls
tf["JoinLeft JoinOperation.Add"] = a.JoinLeft(b, JoinOperation.Add);   

// join full combines both time series
// note that nulls will be evaluated to null
tf["JoinFull r + l"] = a.JoinFull(b, (left, right) => left + right); 
// Use pre defined JoinOperation to ignore nulls
tf["JoinFull JoinOperation.Add"] = a.JoinFull(b, JoinOperation.Add);

tf

2020-12-31 23:00:00Z,1,<null>,2,<null>,1,<null>,1
2021-01-01 23:00:00Z,1,<null>,2,<null>,1,<null>,1
2021-01-02 23:00:00Z,1,2,2,3,3,3,3
2021-01-03 23:00:00Z,1,2,2,3,3,3,3
2021-01-04 23:00:00Z,1,2,2,3,3,3,3
2021-01-05 23:00:00Z,<null>,2,<null>,<null>,<null>,<null>,2


### Slicing

In [ ]:
var ts = TimeSeries.Factory.FromGenerator(
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 01, 05), // end
    Period.Day,
    tp => tp.Day); // 1, 2, 3, 4, 5
 
var tf = new TimeFrame();
// sliceing by index / count
tf["ts"] = ts;
// slicing by index / count
tf["Slice(0, 2)"] = ts.Slice(0, 2);
// slicing by time range
tf["Slice(2.1, 4.1)"] = ts.Slice(new DateTime(2021, 01, 02), new DateTime(2021, 01, 04));

tf
 

2020-12-31 23:00:00Z,1,1,<null>
2021-01-01 23:00:00Z,2,2,2
2021-01-02 23:00:00Z,3,<null>,3
2021-01-03 23:00:00Z,4,<null>,4


### Re-sampling

In [ ]:
var ts = TimeSeries.Factory.FromValue(1,
    new DateTime(2021, 01, 01), // start
    new DateTime(2021, 12, 31), // end
    Period.Day);

// down sampling
var tf = new TimeFrame();
tf["Resample Sum"] = ts.ReSample(Period.Month, AggregationType.Sum);
tf["Resample Mean"] = ts.ReSample(Period.Month, AggregationType.Mean); 
tf

2020-12-31 23:00:00Z,31,1
2021-01-31 23:00:00Z,28,1
2021-02-28 23:00:00Z,31,1
2021-03-31 22:00:00Z,30,1
2021-04-30 22:00:00Z,31,1
2021-05-31 22:00:00Z,30,1
2021-06-30 22:00:00Z,31,1
2021-07-31 22:00:00Z,31,1
2021-08-31 22:00:00Z,30,1
2021-09-30 22:00:00Z,31,1
2021-10-31 23:00:00Z,30,1


In [ ]:
// up-sampling
var tf = new TimeFrame();
tf["Resample Hour Sum"] = ts.ReSample(Period.Hour, AggregationType.Sum);
tf["Resample Hour Mean"] = ts.ReSample(Period.Hour, AggregationType.Mean);
tf

time Resample Hour Sum Resample Hour Mean 2020-12-31 23:00:00Z 0.0416666666666666666666666667 1 2021-01-01 00:00:00Z 0.0416666666666666666666666667 1 2021-01-01 01:00:00Z 0.0416666666666666666666666667 1 2021-01-01 02:00:00Z 0.0416666666666666666666666667 1 2021-01-01 03:00:00Z 0.0416666666666666666666666667 1 ... 8749 more 2021-12-31 17:00:00Z 0.0416666666666666666666666667 1 2021-12-31 18:00:00Z 0.0416666666666666666666666667 1 2021-12-31 19:00:00Z 0.0416666666666666666666666667 1 2021-12-31 20:00:00Z 0.0416666666666666666666666667 1 2021-12-31 21:00:00Z 0.0416666666666666666666666667 1

---
## .Net Interactive Integration

In [ ]:
var rnd = new Random();
var start = new DateTime(2021, 01, 01);
var end = new DateTime(2021, 12, 31);
var tf = new TimeFrame();
var prev = (decimal?)0M;

decimal? ValueGenerator(DateTimeOffset ts)
{
    prev += (decimal)Math.Round((rnd.NextDouble() - 0.5), 4);
    return prev;
} 

for (int i = 0; i < 10; i++)
{
    prev = (decimal?)0M;
    tf.Add("ts_" + i, TimeSeries.Factory.FromGenerator(start, end, Period.Hour, ValueGenerator));
}

tf

time ts_0 ts_1 ts_2 ts_3 ts_4 ts_5 ts_6 ts_7 ts_8 ts_9 2020-12-31 23:00:00Z 0.4997 -0.0296 -0.3145 -0.1179 -0.2094 0.0954 0.0047 0.3907 -0.0998 -0.4659 2021-01-01 00:00:00Z 0.1047 -0.1879 -0.5164 -0.3107 0.2649 -0.2240 -0.2080 0.5940 -0.2278 -0.8412 2021-01-01 01:00:00Z -0.3886 -0.1507 -0.5607 -0.4583 -0.0222 0.0420 0.1645 1.0926 -0.5664 -0.6684 2021-01-01 02:00:00Z -0.8356 -0.5758 -0.9410 -0.4872 -0.3636 -0.2554 -0.0951 0.7846 -0.2810 -0.3071 2021-01-01 03:00:00Z -0.6236 -0.9334 -0.8899 -0.0465 -0.4518 -0.7078 -0.3621 0.3246 -0.5430 -0.2944 ... 8726 more 2021-12-30 18:00:00Z 17.0310 4.3861 21.7229 -25.8094 28.5696 30.3946 14.9002 -3.5921 16.4292 -32.6256 2021-12-30 19:00:00Z 17.3515 4.5294 22.1532 -26.0825 28.2064 30.7659 14.4590 -3.8077 16.1059 -32.6505 2021-12-30 20:00:00Z 17.7082 4.8692 22.5397 -26.3847 28.5209 31.0795 14.8723 -3.3429 16.5145 -32.7158 2021-12-30 21:00:00Z 17.5626 4.6005 23.0174 -26.0282 28.6387 30.8909 14.4283 -2.9588 16.3520 -33.0799 2021-12-30 22:00:00Z 17.3157 4.7019 22.6103 -26.3357 28.7775 30.6485 14.1698 -3.3862 16.0297 -32.9071

In [ ]:
tf.Plot(chartTitle: "Random Walk", xAxisTitel: "Time (hourly)", yAxisTitle: "Value")

<!DOCTYPE html>